# DSCI6003 4.3 Lecture - Advanced Trees and Model Construction

## By the End of this Lecture You Will:
1. Be familiar with regression trees
2. Be cursorily familiar with various tree (and forest) construction algorithms
3. Know the basic pruning algorithm
4. Be familiar with regression trees
5. Become familiar with bagging other models
6. Be familiar with the construction of test harnesses and pipelines

## Regression Trees

Like many - if not most - standard classifiers, trees and by extension, forests, can be converted into regressors and thus used for continuous predictions. In the case of single trees, the tree parses the inputs into large domains, and produces a single output (based on the training data) as an output of that domain (where the domain is defined by the splits).

In the below figures ([obtained from lecture notes here](http://www.stat.cmu.edu/~cshalizi/350-2006/lecture-10.pdf)) we see the construction of trees for predicting car price. Below is a standard tree regressed against horsepower and wheelbase: 

![tree1](./images/Wheelbase_tree_2.png)

The following figure is typically called a *partititioning* of the data, where the boundaries of each domain are determined based on the branch points of the tree. When the regression tree is built, the y_train values of each member of the leaf splits are averaged to create a single output. Although this partition is visualizable in two dimensions, additional regressor variables create further dimensions to the partition:

![partition](./images/Tree_Parsing_1.png)


It's worth noting that if multiple predictor variables (features) are equally good at determining the outcome, the split chosen at that branch level is determined by chance. This is the essential concept behind the notion of *feature importance*. Here is the same tree, regressed against weight instead of wheelbase at the second level:


![tree1](./images/Weight_tree_1.png)


#### Regression Forests

Just as we do with random forest, a regression forest simply averages the predictions of its component trees (thus averaging a lot).

### The Pruning Algorithm 

#### Prepruning:

There are two essential ways of pruning a tree: **pruning** and **prepruning**. We call it **prepruning** if we impose (manually input) stopping criteria on the tree while it grows. This helps limit the number of leaf-node relationships we have to traverse after the tree is built. However, the criteria that are used to perform prepruning are determined by the user and so are subject to user expertise. 

    Construct root node:
        while X is larger than one row or not preprune:
            find set S = {S1, S2} that minimizes impurities
            choose the S that minimizes the size
            construct child nodes and pass them S
            check preprune 
      
        check preprune:
            if tree has reached max depth or ratio of splits is imbalanced or information gain < error:
                 preprune = True
                 
                 
#### Pruning:

After the construction of the tree is complete, we go back and pass the tree into a prune algorithm. The pruning algorithm simply merges leaves if they improve the error rate of the tree. It works recursively, allowing as many splits as necessary.

    While not a root node or not a singleton leaf:
        use the current tree to predict y
        tentatively merge classes (training y) of this split
        if the accuracy of the merge ( sum(merged_y==y_train)/float(y.shape[0])) is greater than the unmerged
            set the current node to be a leaf
            set the current classes to be the merged classes
            set the current node name to be the merged name
            delete left and right leaves.

There have been alternative ways proposed to prune a tree, but this is the most common.

## Construction of Advanced Trees

### Industrial Tree Algorithms and their Outputs

Decision Trees (and by extension Random Forests) are still being researched, and so there are a number of construction algorithms for trees in circulation today. 

In truth, many classifiers in use today share this characteristic in common - they can be improved on, augmented, or otherwise enhanced. ** You should consider it part of your portfolio of professional skills to keep abreast of developments in all algorithms you claim to have expertise in.**

### Standard algorithms: CART and C4.5

The first methods of constructing trees were developed in the early 70's by statisticians studying the analysis of survey answers. You have already built an algorithm that closely follows the original:

    Construct root node
        while X is larger than one row
            find set S = {S1, S2} that minimizes impurities
            choose the S that minimizes the size
            construct child nodes and pass them S

The same basic algorithm is used in more modern methods, CART and C4.5, follow a similar approach, although 10-fold cross validation is used in CART. C4.5 uses a [heuristic approach](https://en.wikipedia.org/wiki/Heuristic) to validate error rates. CART uses the Gini index and C4.5 uses entropy to detect purity. In both cases, however they grow a full tree and then **prune** the tree back to reduce overfitting.



### Newer algorithms: CRUISE, GUIDE, QUEST 

These newer (and far less common) algorithms differ from each other in that they apply significance tests before construction of child nodes. Before assigning a split, each X is tested for association with y and the most significant variable is selected. Then, an exhaustive search is performed for the split S.

Every X has the same chance to be selected if each is independent of Y, therefore this method is less prone to selection bias (inadequate randomization at each node). 

The tree is **pruned** the same as CART.

    Construct root node
        while X is larger than one row
           determine if X is ordered
           convert X to an unordered variable X' by grouping values into a small number of intervals
           perform chi-squared test of each X' vs y and compute significance
           choose X* in X' that has the smallest significance with y
           Find the split S* that minimizes Gini index
           construct child nodes and pass them S*


# Bagging Other Models

Although bagging is the primary method by which we enhance decision trees (Random Forests), bagging is essentially a universal approach to improving models and virtually any other model can be bagged. 

Cross-validation is often used to check the robustness of a model. In this case, we can think of bagging as a sort of "cross-construction" in order to produce a more robust model than would otherwise be possible (less variance at the cost of some additional bias).

If a large component of sampling error is expected in the data set (or there are flaws in the sampling), bagging can be a good way of reducing model variance and overfitting, and is worthy to try in order to provide some validation to an otherwise shaky model. 

Bagging is not a magic power however, and cannot solve the weaknesses of a bad estimator choice or terrible data. 

Scikit-learn has a good implementation for bagging both [classifiers](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.BaggingClassifier.html) and [regressors](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.BaggingRegressor.html). In both these cases one passes the estimator class as a first class argument into the bagging function and it generates n_estimators bootstrap samples from the original.


## QUIZ:
Give a case example where you might bag a model.

# Test Harnesses

One phrase you will hear around our campus that you may not hear everywhere (but most professionals will understand it anyway) is "test harness." A "test harness" is an organized way of gathering data about an estimator (newly coded or otherwise) so that you have some idea of what its modeling qualities are before you haul off and try to apply it somewhere. There are typically two things that are included in a test harness (and you need both):

1. Series of unit tests to ensure that the output of methods and classes within newly-written or -modified code is both predictable and correct.
2. Series of short-form experiments on a controlled data sample (you must pick) in order to gather useful performance metrics (F-tests and confusion matrices).

The former should be something that you continuously update as you write newly coded functions, and should be a main part of your consideration as you go. This means that every time you write a new function, you should be thinking about what the unit test for this might be, and have a very good idea of what it should output.

The latter should be constructed when you are done passing unit tests, and usually is a separate script. You need to make informed choices about what other estimators to compare yours to, and which performance metrics are actually valid in your case. 

## QUIZ:

How might we construct a test harness for the entropy function, $H = \sum_{i} P(c_{i}) log{P(c_{i})}$?


# Pipelines, Grid Searches and Batch Jobs


### Pipelines

You have heard me use the word "pipeline" quite a bit already. Not all data scientists use this word, it is very "engineering" In the most general sense a pipeline is simply a chain of modelling tasks that one might perform on the way to getting a professional quality model. 

In the first use case, pipelines are a way of constructing a model from a group of estimators where each estimator acts as a **transform** to the next. For example, one might construct a pipeline where the data is first clustered with one method, then each cluster is studied with a linear model (I actually just did this). If we wanted to create a program that regularly, we would construct a pipeline for doing so as part of the program. Scikits has a [useful example of this you can play with](http://scikit-learn.org/stable/auto_examples/model_selection/grid_search_text_feature_extraction.html#example-model-selection-grid-search-text-feature-extraction-py). 

Obviously, the types of outputs of each estimator involved are quite important to pipeline construction.

### Grid Searches

Grid searches are typically used when you don't know (and often don't care too much about the meaning of) a set of optimal parameters to a given estimator or set of estimators. They are essentially a set of for loops that try out a series of parameters and construct a single model for each case (hence a grid). Scikits has a [grid search class](http://scikit-learn.org/stable/modules/grid_search.html#grid-search) that will automate an exhaustive or optimized search for one or more estimator parameters. 

Also somewhat confusingly, people will often conflate "pipeline" and "grid search", sometimes using the former to mean the latter. You can do a grid search as part of a pipeline, using a final function to estimate model quality, and the output of the tested models as input. Scikits has an [example of this here.](http://scikit-learn.org/stable/modules/pipeline.html#pipeline). You can also see the Feature Union class on this page, which is a way of constructing boosted models (we will cover soon).

### Batch jobs

Batch jobs are usually done when the final model includes many different database queries, aggregations, constructions of estimators, grid searches, pipelines and final output plots. Imagine all of these as separate tasks, where individual lists of commands with all their different parameters might number in the hundreds or thousands. For example if we needed to grid search a model over a huge query space. Sometimes this process is also called a "Pipeline." Some professional software has been [written to automate this process](https://pypi.python.org/pypi/luigi) as well. It is less common to find this level of sophistication in the wild, but it's out there.

## QUIZ:

Imagine examples where we might want to use the above methods.